In [46]:
#We need these in this file:
import pandas as pd
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score
import numpy as np # linear algebra
import get_lodging_scores
import load_read_name_extractor as lrne
import SVM_classifier_general as svm_general
import pickle
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV

In [47]:
data_paths = [
"Features\hog_features_(64, 128, 3)_cells_(8, 8)_block_(1, 1)_norm_L2.npy",
"Features\hog_features_(64, 128, 3)_cells_(8, 8)_block_(2, 2)_norm_L2.npy",
"Features\hog_features_(64, 128, 3)_cells_(8, 8)_block_(3, 3)_norm_L2.npy",
"Features\hog_features_(64, 128, 3)_cells_(8, 8)_block_(4, 4)_norm_L2.npy",
"Features\hog_features_(96, 192, 3)_cells_(8, 8)_block_(1, 1)_norm_L2.npy",
"Features\hog_features_(96, 192, 3)_cells_(8, 8)_block_(2, 2)_norm_L2.npy",
"Features\hog_features_(96, 192, 3)_cells_(8, 8)_block_(3, 3)_norm_L2.npy",
"Features\img_names_(64, 128, 3)_cells_(8, 8)_block_(1, 1)_norm_L2.npy",
"Features\img_names_(64, 128, 3)_cells_(8, 8)_block_(2, 2)_norm_L2.npy",
"Features\img_names_(64, 128, 3)_cells_(8, 8)_block_(3, 3)_norm_L2.npy",
"Features\img_names_(64, 128, 3)_cells_(8, 8)_block_(4, 4)_norm_L2.npy",
"Features\img_names_(96, 192, 3)_cells_(8, 8)_block_(1, 1)_norm_L2.npy",
"Features\img_names_(96, 192, 3)_cells_(8, 8)_block_(2, 2)_norm_L2.npy",
"Features\img_names_(96, 192, 3)_cells_(8, 8)_block_(3, 3)_norm_L2.npy"]

In [48]:
features_paths = data_paths[0:7]
names_paths = data_paths[7:14]

In [49]:
def save_results(results, path):
    path = path.replace("Features\\", "")
    path = "results/" + path.replace(".npy", ".txt")
    with open(path, 'a') as f:
        f.write(results)
        f.write("\n")

In [50]:
def load_labels(bin_size):
    labels = get_lodging_scores.get_labels(bin_size)
    #COnverts into set (no dublicates in set)
    my_set = set(map(tuple, labels))
    # convert set back to list of arrays
    labels = list(map(np.array, my_set))
    return labels

In [51]:
def partition_data(data_frame, features_aar, percentage):
    partition = int(len(features_aar)*percentage/100)
    x_train, x_test = data_frame[:partition,:-1],  data_frame[partition:,:-1]
    y_train, y_test = data_frame[:partition,-1:].ravel() , data_frame[partition:,-1:].ravel()
    return x_train, x_test, y_train, y_test

In [52]:
def save_model(filename, clf):
    filename = filename.replace("Features\\", "")
    filename = "model/" + filename.replace(".npy", ".sav")
    pickle.dump(clf, open(filename, 'wb'))

In [53]:
def run_mult_models(features_paths, names_paths, bin_size = 10, svm_kernel="rbf"):
    for i in range (len(features_paths)):
        #laod features and names
        features_load = np.load(features_paths[i], allow_pickle=True)
        names_load = np.load(names_paths[i], allow_pickle=True)

        #load labels
        labels = load_labels(bin_size)

        #convert names into [flight_folder, ROI]
        names = lrne.it_name_extract_labels_from_img_jpeg(names_load)

        #create svm
        clf = svm.SVC(kernel=svm_kernel)

        #match labels with feature names
        data_frame, error = svm_general.match_pic_label_to_names(features_load, labels, names)
        #data_frame_names, error_names = match_pic_label_names(features_load, labels, names) - finds images that has no labels

        #shuffle data and convert to array
        np.random.shuffle(data_frame)
        data_frame = np.array(data_frame)

        #partition data
        percentage = 80
        features_aar = np.array(features_load)
        x_train, x_test, y_train, y_test = partition_data(data_frame, features_aar, percentage)

        #fit model with training data
        clf.fit(x_train,y_train)

        #predict data
        y_pred = clf.predict(x_test)

        #Save the found model
        n_bins = int(np.floor(100/bin_size))
        class_weights = "none" #removed
        path = "Kernel_" + svm_kernel + "_Nbins_" + str(n_bins) + "_ClassWeights_" + class_weights + "_" + features_paths[i]
        save_model(features_paths[i], clf)

        #Save the results
        results = "Accuracy: "+str(accuracy_score(y_test, y_pred)) + "\n" + classification_report(y_test, y_pred)
        save_results(results, path)
        
        
    

In [54]:
run_mult_models(features_paths, names_paths, bin_size = 10, svm_kernel="rbf")

c:\Users\marti\miniconda3\envs\imgp2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\marti\miniconda3\envs\imgp2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\marti\miniconda3\envs\imgp2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\marti\mini

In [ ]:
features_load = np.load('features/*', allow_pickle=True)
names_load = np.load('features/img_names_64_128_hog_cells_8x8_block_2x2.npy', allow_pickle=True)
print(len(features_load))
print(len(names_load))

In [10]:
labels = get_lodging_scores.get_labels(33)
#[ROI, Visual SCORE, FLIGHTFolder, BIN]
print(features_load[0])
print(labels[0])

[0.20888097 0.16330611 0.04696022 ... 0.06301632 0.05749686 0.03150874]
(61, 50, '160607_Min', 2.0)


In [11]:
my_set = set(map(tuple, labels))
# convert set back to list of arrays
labels = list(map(np.array, my_set))

In [12]:
print(len(labels))

4936


In [13]:
names = lrne.it_name_extract_labels_from_img_jpeg(names_load)

In [14]:
clf = svm.SVC(kernel="rbf", class_weight="balanced")
features_aar = np.array(features_load)
data_frame, error = SVM_GENERAL.match_pic_label_to_names(features_load, labels, names)
#data_frame_names, error_names = match_pic_label_names(features_load, labels, names) - finds images that has no labels
np.random.shuffle(data_frame)
data_frame = np.array(data_frame)


In [15]:
percentage = 80
partition = int(len(features_aar)*percentage/100)

In [16]:
x_train, x_test = data_frame[:partition,:-1],  data_frame[partition:,:-1]
y_train, y_test = data_frame[:partition,-1:].ravel() , data_frame[partition:,-1:].ravel()
print(y_train)

clf.fit(x_train,y_train)

['1.0' '1.0' '1.0' ... '2.0' '1.0' '1.0']


SVC(class_weight='balanced')

In [17]:
print(clf)

SVC(class_weight='balanced')


In [18]:
y_pred = clf.predict(x_test)

In [19]:
filename = 'SVM_linear_64_128_3bins_cells_8x8_block_2x2_balanced.sav'
pickle.dump(clf, open(filename, 'wb'))

In [20]:
#loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, Y_test)

In [21]:
print("Accuracy: "+str(accuracy_score(y_test, y_pred)))
print('\n')
print(classification_report(y_test, y_pred))

Accuracy: 0.717683557394002


              precision    recall  f1-score   support

         1.0       0.86      0.80      0.83       687
         2.0       0.39      0.52      0.45       192
         3.0       0.60      0.50      0.55        88

    accuracy                           0.72       967
   macro avg       0.62      0.61      0.61       967
weighted avg       0.74      0.72      0.73       967

